# Twitter's Climate Tide: An Analysis of Climate Change Sentiment on Twitter
### By Arjun Gandhi
### Last updated: December 10, 2020
### THIS IS A DRAFT.

# Data Collection
I am starting off with a data set from Harvard that contains 39.6 million tweets related to climate change. The data set is in tweet IDs (numbers) so I need get the tweets for each tweet ID.

Here is the link the data set: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/5QCCUU

As states in the above link the data is from September 21, 2017 and May 17, 2019 and they had a gap in data collection from January 7, 2019 to April 17, 2019.

To convert each tweet ID into the actual tweet data I am using this: Hydrator [Computer Software]. Retrieved from https://github.com/docnow/hydrator

From the above repo, I downloaded this version of the app: https://github.com/DocNow/hydrator/releases/tag/v0.0.13

The tweets are seperated by file (~ 10 million tweets/file). I made a Twitter account to connect my account this Hydrator. I then uploaded each txt file into Hydrator under "Datasets" in the desktop app. 

TALK ABOUT THEIR METHODOLOGY

In [1]:
import pandas as pd

data = pd.read_csv("./data/tweets200k.csv")
data = data.head(5000) # REMOVE THIS LINE !!!!!!!!!!!!!!
data.head()

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified
0,NaN,Tue Dec 11 01:00:00 +0000 2018,UN cdnpoli ONpoli ABpoli,https://twitter.com/TheRebelTV/status/10722948...,https://www.therebel.media/un-global-warming-m...,92,1072294898588631040,NaN,NaN,NaN,...,203482,17522,1243,Canada and the world,Rebel News,RebelNewsOnline,38905,NaN,https://www.rebelnews.com,True
1,NaN,Mon Jan 22 09:49:35 +0000 2018,NaN,NaN,NaN,0,955376892026093569,Pontifex,9.551606e+17,500704345.0,...,1,4,0,United States,Frank,Frank34802901,100,NaN,NaN,False
2,NaN,Mon Sep 17 04:42:16 +0000 2018,NaN,NaN,https://truthout.org/articles/national-park-of...,0,1041547863795224576,NaN,NaN,NaN,...,2065,2383,98,USA,OurRevolution,LeftysUnite,49972,NaN,NaN,False
3,NaN,Sat Aug 04 13:02:13 +0000 2018,Spain Portugal climatechange globalwarming Hea...,NaN,https://news.sky.com/story/live-scorching-satu...,1,1025728615399469058,NaN,NaN,NaN,...,25,24,1,NaN,Steven Hugh,Steven9Hugh,1402,NaN,https://stevenhugh.wordpress.com/,False
4,NaN,Tue Nov 21 10:17:51 +0000 2017,Resist FakePresident Dontard GOP NRA War Clima...,NaN,https://twitter.com/mattmfm/status/93272970237...,0,932915956824682496,NaN,NaN,NaN,...,6191,6720,121,"the beautiful ""Jemez"" USA",Athoughtz,athoughtz,155930,NaN,http://TokTok.com,False


# Data Wrangling
The data set has lots of data that is not needed for this analysis. Since we are looking at sentiment over time and other factors related to polticis of a state and events, it is simplest to just drop all non-English tweets.

There are lots of extranenous columns that are not relavent to this project so I just dropped them. These include things like user specifics like their profile details and other things like the URL of thr tweet or the language since all will be English. 

I then renamed some columns for my ease of use of the data set and switched the tweet ID to be the index columns.

The date and time is given as a string so I use regular expressions to convert that to a date time object. The hashtag column is given as one string so I split that into a list of hastags.

In [2]:
# Remove non-English tweets from the data set
data = data[data["lang"] == "en"]

# Drop all the unneeded columns from the data set
cols_to_delete = ["user_urls", "user_statuses_count", "coordinates", "user_name", "in_reply_to_status_id", 
                  "in_reply_to_user_id", "user_time_zone", "urls", "lang", "media", "source", 
                  "retweet_screen_name", "retweet_id", "possibly_sensitive", "tweet_url",
                  "user_default_profile_image", "user_friends_count", "user_verified", "user_location", 
                   "in_reply_to_screen_name", "user_screen_name.1",
                  "user_favourites_count", "user_listed_count", "user_created_at", "user_description", "place", 
                 "user_followers_count", "hashtags", "favorite_count", "retweet_count"]

data = data.drop(columns=cols_to_delete)

# Swap the index column from 0...n to the tweet ID and rename the column from id to tweetID and rename to clarify
# column meaning
data = data.rename(columns={"id": "tweetID", "created_at": "date/time", "user_screen_name": "tweeter"})
data = data.set_index('tweetID')

In [3]:
# Convert the dates time strings into datetime objects
import re
import datetime

dates = []

# Matching this text
# Mon Jan 22 09:49:35 +0000 2018
# For every row in the dataframe
regex = re.compile(r"(\w{3}) (\w{3}) (\d\d) (\d\d:\d\d:\d\d) \+(0{4}) (\d{4})")

# Given a string of a month return the corresponding integer for that month i.e. Jan == 1
def numerize(str):
    month = str.lower()
    if (month == "jan"): return 1
    elif (month == "feb"): return 2
    elif (month == "mar"): return 3
    elif (month == "apr"): return 4
    elif (month == "may"): return 5
    elif (month == "jun"): return 6
    elif (month == "jul"): return 7 
    elif (month == "aug"): return 8
    elif (month == "sep"): return 9
    elif (month == "oct"): return 10
    elif (month == "nov"): return 11
    elif (month == "dec"): return 12
        
for row in data.iterrows():
    dt = row[1]["date/time"]
    matches = re.search(regex, dt)
    groups = matches.groups()    
    month = numerize(groups[1])
    d = datetime.date(int(groups[5]), month, int(groups[2]))
    dates.append(d)
    
data = data.drop(columns=["date/time"])
data["date_tweeted"] = dates

## Prepare the tweets for sentiment analysis
There are several things that need to be done to the actual text of the tweets before we can do sentiment analysis on them. To starts of, I will do some basic things like make all tweet bodies lower case so that words like CLIMATE and climate and cLiMate are all treated the same by the model I use later on. Next, I am going to remove all links from these tweets because that is irrelvanet to the sentiment of the tweet. There are many things like this that I will do here then I will move on to make the tweets "linguistically sound" for analysis by doing things like removing words without meaning that won't contribute the analysis and then making all words their base word or lemmatizing them.

### Make all tweets lower case

In [4]:
# Make all tweets lower case so that when use a model to look at sentiment words that are the same are treated
# so by the model i.e. capitalization won't make the model think Word is not word.
lower_case_tweets = []
for r in data.iterrows(): lower_case_tweets.append(r[1]["text"].lower())
data = data.drop(columns=["text"])
data["text"] = lower_case_tweets

### Remove links from tweets
Links are pretty useless if I want to know what the text of a tweet means. I could have gone a step further and taken the links and scraped it and then looked at that sentiment but for simplicty I omitted this step.

In [5]:
linkless = []
regex = re.compile(r"http\S+")

# remove all links from each tweet
for row in data.iterrows():
    txt = row[1]["text"]
    if txt.find("https://t.co"): 
        ll = re.sub(regex, "", txt)
        linkless.append(ll)
    else: 
        linkless.append(txt)

data["text"] = linkless

### Remove puncuation and digits
Puncuation does not really add much when doing sentiment analysis and so I will remove puncuation from the tweets. I will also remove tokens (words or digits) in the tweets that contain digits so words with a digit in it or a digit from the tweet for similar reasons.

In [6]:
# Referenced: https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
import string

# Remove all puncuation
data['text']=data['text'].apply(lambda tweet: tweet.translate(str.maketrans('', '', string.punctuation)))

# Remove digits
data['text']=data['text'].apply(lambda tweet: re.sub("\w*\d\w*","", tweet))

### Stopwords and Lemmatization 
Here I will remove stopwards from the tweet bodies. These are words like "I" and "this" that add little meaning to the tweet but if left in the text will give me an innacurate depiction of the most common words in the tweets. Thne I will perform lemmatization on the tweets. This just means taking words that linguisticlly mean the same thing like walker and walking and reducing them to their base. In this case the word walk. You can read more here: https://en.wikipedia.org/wiki/Lemmatisation. I will be doing this uisng spaCy (https://spacy.io).

You can find installation instructions for spaCy here: https://spacy.io/usage. 

In [ ]:
# I did these commands to setup spaCy.
!pip install -U spacy
!pip install -U spacy-lookups-data
!python -m spacy download en_core_web_sm

In [8]:
import spacy

# Load the model 
model = spacy.load('en_core_web_sm')

# Lemmatization of tweets and stopwords removal
data['text']=data['text'].apply(lambda t: ' '.join([tok.lemma_ for tok in list(model(t)) if (tok.is_stop==False)]))
data

,tweeter,date_tweeted,text
tweetID,,,
1072294898588631040,RebelNewsOnline,2018-12-11,therebeltv go different un conference — medium...
955376892026093569,Frank34802901,2018-01-22,pontifex prayer god amp true father jesu...
1025728615399469058,Steven9Hugh,2018-08-04,red alert spain portugal europe near alltime h...
932915956824682496,athoughtz,2017-11-21,trump gop swamp resist fakepresident dontard g...
1041547806622797824,IndiaGreenBldg,2018-09-17,study green building save billion health amp...
...,...,...,...
1120089046217306112,JillianDURM,2019-04-21,hope folk fabulous demand win big
1069679067065139203,goldenerin,2018-12-03,good thing planet human extinct
946759916425039872,MagnaCartaRules,2017-12-29,cillizzacnn global warming advocate fact argue...


# Exploratory Data Analysis and Data Visualization

### Create a bag of words of the tweets
I create a bag of words of the tweets using CountVectorizer from sklearn. This doesn't look like it means anything, it literally just looks like a matrix of 0s...but will be super helpful in making word clouds so we can better visualize these tweets!

In [9]:
# Make a bag of words using sklearn
from sklearn.feature_extraction.text import CountVectorizer 

cv=CountVectorizer(analyzer='word')
d=cv.fit_transform(data['text'])
bag = pd.DataFrame(d.toarray(), columns=cv.get_feature_names())
bag.index=data.index
bag

,aaas,aaasmtg,aarneclimate,aaron,abacus,abandon,abandons,abate,abatement,abbot,...,𝐀𝐧𝐭𝐡𝐫𝐨𝐩𝐨𝐠𝐞𝐧𝐢𝐜,𝐂𝐡𝐚𝐧𝐠𝐞,𝐂𝐡𝐞𝐜𝐤𝐢𝐧𝐠,𝐂𝐥𝐚𝐢𝐦,𝐂𝐥𝐢𝐦𝐚𝐭𝐞,𝐂𝐨𝐧𝐬𝐞𝐧𝐬𝐮𝐬,𝐅𝐚𝐜𝐭,𝐎𝐟,𝐎𝐧,𝐓𝐡𝐞
tweetID,,,,,,,,,,,,,,,,,,,,,
1072294898588631040,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
955376892026093569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1025728615399469058,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
932915956824682496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1041547806622797824,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120089046217306112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1069679067065139203,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
946759916425039872,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
